In [1]:
#sample.do
#!pip install linearmodels
#!pip install econtools
import pandas as pd
import numpy as np
import matplotlib as plt
from linearmodels import PanelOLS
import statsmodels.api as sm
import econtools as econ
import econtools.metrics as mt

df = pd.read_stata('data/Authority.dta')
print(df.shape)
#list(df.columns)

#construct work category dummy
df['OG03_dummy'] = 0
df.loc[(df['work_category']=='OG03')&(df['work_category']!=''),'OG03_dummy'] = 1

df['OG01_dummy'] = 0
df.loc[(df['work_category']=='OG01')&(df['work_category']!=''),'OG01_dummy'] = 1

df['OG_rest_dummy'] = 0
df.loc[(df['OG01_dummy']!=1)&(df['OG03_dummy']!=1)&(df['work_category']!=''),'OG_rest_dummy'] = 1

df['OG_dummy'] = 0
df.loc[df['work_category'].str[0:2] == 'OG','OG_dummy'] = 1

df['OS_dummy'] = 0
df.loc[df['work_category'].str[0:2] == 'OS','OS_dummy'] = 1

#treated vs controls
df['trend'] = df['year'] - 1999

df['trend_treat'] = df['trend']
df.loc[(df['authority_code']!=3090272)&(df['authority_code']!=3070001),'trend_treat'] = 0
#15225 real change made / no zeros in df['trend']
#print(df['trend_treat'].value_counts()) chekced

df['trend_control'] = df['trend']
df.loc[(df['authority_code']==3090272)|(df['authority_code']==3070001),'trend_control'] = 0
#902 real change made
#print(df['trend_control'].value_counts()) checked

#PA specifics
df = df.sort_values(by='authority_code',ascending=True)
#auth = econ.group_id(df, cols = 'authority_code')
#print(auth) #dataframe

auth_list = df['authority_code'].values.tolist()
auth_list = list(set(auth_list))
#겹치는 부분 = authority_code야
#id_auth = group_id + 1 if df의 auth code == auth의 code

df['id_auth'] = 0
for i in range(len(df)):
    for j in range(len(auth_list)):
        if df.loc[i,'authority_code'] == auth_list[j]:
            df.loc[i,'id_auth'] = j+1

work_dum = pd.get_dummies(df['work_category'])
year_dum = pd.get_dummies(df['year'])
work_list = list(work_dum.columns)
year_list = list(year_dum.columns)

df_dum = pd.concat([year_dum, work_dum],axis = 1)
df = pd.concat([df, df_dum],axis = 1)

#for statement check            
#df.head
#print(max(df['id_path']))

(16127, 31)


In [7]:
#preserve

df['post01pre05'] = 0
for i in range(len(df)):
    if df.loc[i,'year'] >= 2001 and df.loc[i,'year'] <= 2005 and pd.isnull(df.loc[i,'year']) == False:
        df.loc[i,'post01pre05'] = 1
        
df['post02pre04'] = 0
for i in range(len(df)):
    if df.loc[i,'year'] >= 2002 and df.loc[i,'year'] <= 2004 and pd.isnull(df.loc[i,'year']) == False:
        df.loc[i,'post02pre04'] = 1

df['complexity_dummy'] = 0
for i in range(len(df)):
    if df.loc[i,'complex_work'] ==2 or df.loc[i,'complex_work']==3:
        df.loc[i, 'complexity_dummy'] = 1

In [ ]:
complex_list = df['year'].values.tolist()
complex_list = list(set(complex_list))
print(complex_list)

In [ ]:
#table 6
#dependent var = discount
#(:,1)/ co& ctrl_exp

treatment = ['turin_co_sample','turin_pr_sample']
# mutlicollinearity function
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

#iteration for group
for t in treatment:
    df_reg_co = df[(df[t]==1)&(df['ctrl_exp_' + t]==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
    
    #vif cal
    #first, make a column list
    reg_col = []
    for i in work_list:
        reg_col.append(i)
    for j in year_list:
        reg_col.append(j)
    exog_var = ['fpsb_auction','reserve_price','municipality'] #id_auth빼봄
    exog = exog_var + reg_col 


    #check multicollinearity
    X = df_reg_co.loc[:,exog]
    vif = calc_vif(X)
    #print(vif)


    #delete from col list
    for i in range(len(vif)):
        if np.isnan(vif.loc[i, 'VIF']) == True:
            reg_col.remove(vif.loc[i, 'variables'])
        elif vif.loc[i,'VIF'] > 10:
            for j in exog_var:
                if str(vif.loc[i,'variables']) is j and vif.loc[i,'variables'] is not 'fpsb_auction' and vif.loc[i,'variables'] is not 'id_auth':
                    exog_var.remove(vif.loc[i,'variables'])
                
    exog = exog_var + reg_col
    #exog.remove('id_auth')
    exog.remove(2000)
    exog.remove('OG01')
    exog.remove('municipality')

    #print(exog) #check
    print(mt.reg(df_reg_co, 'discount', exog, fe_name = 'authority_code', cluster = 'auth_anno'))

In [ ]:
#table 6
#dependent var = discount
#(:,2)/ co& ctrl_exp

treatment = ['turin_co_sample','turin_pr_sample']
# mutlicollinearity function
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

#iteration for group
for t in treatment:
    df_reg_co = df[(df[t+'_full']==1)&(df['ctrl_exp_' + t]==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
    
    #vif cal
    #first, make a column list
    reg_col = []
    for i in work_list:
        reg_col.append(i)
    for j in year_list:
        reg_col.append(j)
    exog_var = ['fpsb_auction','reserve_price','municipality'] #id_auth빼봄
    exog = exog_var + reg_col 


    #check multicollinearity
    X = df_reg_co.loc[:,exog]
    vif = calc_vif(X)
    #print(vif)


    #delete from col list
    for i in range(len(vif)):
        if np.isnan(vif.loc[i, 'VIF']) == True:
            reg_col.remove(vif.loc[i, 'variables'])
        elif vif.loc[i,'VIF'] > 10:
            for j in exog_var:
                if str(vif.loc[i,'variables']) is j and vif.loc[i,'variables'] is not 'fpsb_auction' and vif.loc[i,'variables'] is not 'id_auth':
                    exog_var.remove(vif.loc[i,'variables'])
                
    exog = exog_var + reg_col
    #exog.remove('id_auth')
    exog.remove(2000)
    exog.remove('OG01')
    exog.remove('municipality')

    #print(exog) #check
    print(mt.reg(df_reg_co, 'discount', exog, fe_name = 'authority_code', cluster = 'auth_anno'))

In [ ]:
#table 6
#(:,3)/ co& ctrl_exp

treatment = ['turin_co_sample','turin_pr_sample']
# mutlicollinearity function
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

#iteration for group
for t in treatment:
    df_reg_co = df[(df[t]==1)&(df['ctrl_exp_'+t] == 1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==1)]
    
    #vif cal
    #first, make a column list
    reg_col = []
    for i in work_list:
        reg_col.append(i)
    for j in year_list:
        reg_col.append(j)
    exog_var = ['fpsb_auction','reserve_price','municipality'] #id_auth빼봄
    exog = exog_var + reg_col 


    #check multicollinearity
    X = df_reg_co.loc[:,exog]
    vif = calc_vif(X)
    #print(vif)


    #delete from col list
    for i in range(len(vif)):
        if np.isnan(vif.loc[i, 'VIF']) == True:
            reg_col.remove(vif.loc[i, 'variables'])
        elif vif.loc[i,'VIF'] > 10:
            for j in exog_var:
                if str(vif.loc[i,'variables']) is j and vif.loc[i,'variables'] is not 'fpsb_auction' and vif.loc[i,'variables'] is not 'id_auth':
                    exog_var.remove(vif.loc[i,'variables'])
                
    exog = exog_var + reg_col
    #exog.remove('id_auth')
    exog.remove(2000)
    exog.remove('OG01')
    exog.remove('municipality')

    #print(exog) #check
    print(mt.reg(df_reg_co, 'discount', exog, fe_name = 'authority_code', cluster = 'auth_anno'))

In [14]:
#dummy value counts
#all checked
df['complexity_dummy'].value_counts()
df['post01pre05'].value_counts()
df['post02pre04'].value_counts()

0    10312
1     5815
Name: post02pre04, dtype: int64

In [15]:
#table 6
#(:,4)/ co& ctrl_exp
#to be checked
#n = 706/ 744
#dummy = 1일때(col5)는 문제 없음

treatment = ['turin_co_sample','turin_pr_sample']
# mutlicollinearity function
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

#iteration for group
for t in treatment:
    df_reg_co = df[(df['complexity_dummy']==0)&(df[t]==1)&(df['ctrl_exp_'+t] == 1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
    #vif cal
    #first, make a column list
    reg_col = []
    for i in work_list:
        reg_col.append(i)
    for j in year_list:
        reg_col.append(j)
    exog_var = ['fpsb_auction','reserve_price','municipality'] #id_auth빼봄
    exog = exog_var + reg_col 


    #check multicollinearity
    X = df_reg_co.loc[:,exog]
    vif = calc_vif(X)
    #print(vif)


    #delete from col list
    for i in range(len(vif)):
        if np.isnan(vif.loc[i, 'VIF']) == True:
            reg_col.remove(vif.loc[i, 'variables'])
        elif vif.loc[i,'VIF'] > 10:
            for j in exog_var:
                if str(vif.loc[i,'variables']) is j and vif.loc[i,'variables'] is not 'fpsb_auction' and vif.loc[i,'variables'] is not 'id_auth':
                    exog_var.remove(vif.loc[i,'variables'])
                
    exog = exog_var + reg_col
    exog.remove(2000)
    #exog.remove('OG01')
    exog.remove('municipality')
    
    exog.remove('OS24')

    print(exog) #check
    print(mt.reg(df_reg_co, 'discount', exog, fe_name = 'authority_code', cluster = 'auth_anno'))

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


['fpsb_auction', 'reserve_price', 'OG03', 'OG08', 'OS01', 'OS07', 'OS08', 'OS10', 'OS12', 2001.0, 2002.0, 2003.0, 2004.0, 2005.0, 2006.0]
Dependent variable:	discount
N:			706
R-squared:		0.6350
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  97
Fixed effects by:	authority_code
  No. of FE:		  15
               coeff    se      t   p>t CI_low CI_high
fpsb_auction  12.743 2.145  5.941 0.000  8.486  17.000
reserve_price  0.000 0.000  6.405 0.000  0.000   0.000
OG03          -2.297 1.703 -1.349 0.181 -5.678   1.084
OG08          -1.417 2.493 -0.568 0.571 -6.365   3.531
OS01          -3.823 2.050 -1.865 0.065 -7.892   0.246
OS07           0.149 2.378  0.063 0.950 -4.572   4.869
OS08           3.755 3.791  0.991 0.324 -3.770  11.280
OS10          13.093 2.415  5.421 0.000  8.298  17.887
OS12          14.789 2.252  6.567 0.000 10.318  19.259
2001.0         3.653 1.594  2.292 0.024  0.490   6.816
2002.0         1.454 1.743  0.834 0.406 -2.007  

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


['fpsb_auction', 'reserve_price', 'OG03', 'OG08', 'OG13', 'OS01', 'OS07', 'OS08', 'OS09', 'OS10', 'OS12', 2001.0, 2002.0, 2003.0, 2004.0, 2005.0, 2006.0]
Dependent variable:	discount
N:			744
R-squared:		0.5783
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  113
Fixed effects by:	authority_code
  No. of FE:		  18
               coeff    se      t   p>t CI_low CI_high
fpsb_auction   9.411 1.786  5.268 0.000  5.872  12.951
reserve_price  0.000 0.000  6.888 0.000  0.000   0.000
OG03           0.568 1.536  0.370 0.712 -2.476   3.612
OG08           4.951 2.444  2.026 0.045  0.108   9.795
OG13           0.055 1.675  0.033 0.974 -3.263   3.372
OS01           0.737 1.819  0.405 0.686 -2.867   4.341
OS07           2.923 1.939  1.507 0.135 -0.920   6.766
OS08           7.389 3.242  2.279 0.025  0.966  13.813
OS09           7.514 1.862  4.036 0.000  3.825  11.202
OS10          15.432 2.065  7.475 0.000 11.341  19.522
OS12          12.675 2.974  4.2

In [22]:
#table 6
#(:,5)/ co& ctrl_exp

treatment = ['turin_co_sample','turin_pr_sample']
# mutlicollinearity function
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

#iteration for group
for t in treatment:
    df_reg_co = df[(df['complexity_dummy']==1)&(df[t]==1)&(df['ctrl_exp_'+t] == 1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
    
    #vif cal
    #first, make a column list
    reg_col = []
    for i in work_list:
        reg_col.append(i)
    for j in year_list:
        reg_col.append(j)
    exog_var = ['fpsb_auction','reserve_price','municipality'] #id_auth빼봄
    exog = exog_var + reg_col 


    #check multicollinearity
    X = df_reg_co.loc[:,exog]
    vif = calc_vif(X)
    #print(vif)


    #delete from col list
    for i in range(len(vif)):
        if np.isnan(vif.loc[i, 'VIF']) == True:
            reg_col.remove(vif.loc[i, 'variables'])
        elif vif.loc[i,'VIF'] > 10:
            for j in exog_var:
                if str(vif.loc[i,'variables']) is j and vif.loc[i,'variables'] is not 'fpsb_auction' and vif.loc[i,'variables'] is not 'id_auth':
                    exog_var.remove(vif.loc[i,'variables'])
                
    exog = exog_var + reg_col
    exog.remove(2000)
    exog.remove('OG01')
    exog.remove('municipality')

    #print(exog) #check
    print(mt.reg(df_reg_co, 'discount', exog, fe_name = 'authority_code', cluster = 'auth_anno'))

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


Dependent variable:	discount
N:			556
R-squared:		0.7091
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  87
Fixed effects by:	authority_code
  No. of FE:		  15
               coeff    se       t   p>t  CI_low CI_high
fpsb_auction  11.077 1.068  10.374 0.000   8.954  13.199
reserve_price  0.000 0.000   4.314 0.000   0.000   0.000
OG02           0.444 0.533   0.833 0.407  -0.616   1.505
OG04          -4.249 3.454  -1.230 0.222 -11.115   2.617
OG06           0.907 0.756   1.200 0.233  -0.596   2.410
OG07           6.435 4.352   1.479 0.143  -2.217  15.087
OG10           7.239 0.572  12.653 0.000   6.101   8.376
OG11           5.353 0.892   6.002 0.000   3.580   7.126
OG12           6.770 1.798   3.765 0.000   3.195  10.344
OS02           8.658 1.212   7.146 0.000   6.249  11.067
OS05           3.294 0.759   4.340 0.000   1.785   4.803
OS06           5.705 1.559   3.659 0.000   2.605   8.805
OS11          -7.666 0.223 -34.332 0.000  -8.110  

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


Dependent variable:	discount
N:			611
R-squared:		0.6395
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  104
Fixed effects by:	authority_code
  No. of FE:		  18
               coeff    se       t   p>t  CI_low CI_high
fpsb_auction   7.529 1.851   4.067 0.000   3.858  11.201
reserve_price  0.000 0.000   5.582 0.000   0.000   0.000
OG02          -0.478 0.565  -0.846 0.399  -1.598   0.642
OG04          -4.648 3.275  -1.419 0.159 -11.143   1.846
OG06           0.029 0.897   0.033 0.974  -1.750   1.809
OG07           3.137 3.236   0.969 0.335  -3.281   9.555
OG09           1.638 1.290   1.270 0.207  -0.921   4.198
OG10           6.930 0.557  12.450 0.000   5.826   8.034
OG11           5.009 0.716   6.992 0.000   3.589   6.430
OG12           4.899 2.398   2.043 0.044   0.143   9.654
OS02           7.796 1.108   7.036 0.000   5.598   9.993
OS05           2.899 0.746   3.888 0.000   1.420   4.378
OS06           2.399 0.982   2.442 0.016   0.451 

In [18]:
#table 6
#(:,6)/ co& ctrl_exp

treatment = ['turin_co_sample','turin_pr_sample']
# mutlicollinearity function
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

#iteration for group
for t in treatment:
    df_reg_co = df[(df['post01pre05']==1)&(df[t]==1)&(df['ctrl_exp_'+t] == 1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
    
    #vif cal
    #first, make a column list
    reg_col = []
    for i in work_list:
        reg_col.append(i)
    for j in year_list:
        reg_col.append(j)
    exog_var = ['fpsb_auction','reserve_price','municipality'] #id_auth빼봄
    exog = exog_var + reg_col 


    #check multicollinearity
    X = df_reg_co.loc[:,exog]
    vif = calc_vif(X)
    #print(vif)


    #delete from col list
    for i in range(len(vif)):
        if np.isnan(vif.loc[i, 'VIF']) == True:
            reg_col.remove(vif.loc[i, 'variables'])
        elif vif.loc[i,'VIF'] > 10:
            for j in exog_var:
                if str(vif.loc[i,'variables']) is j and vif.loc[i,'variables'] is not 'fpsb_auction' and vif.loc[i,'variables'] is not 'id_auth':
                    exog_var.remove(vif.loc[i,'variables'])
                
    exog = exog_var + reg_col
    #exog.remove(2000)
    exog.remove('OG01')
    exog.remove('municipality')

    exog.remove(2005)
    
    #print(exog) #check
    print(mt.reg(df_reg_co, 'discount', exog, fe_name = 'authority_code', cluster = 'auth_anno'))

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


Dependent variable:	discount
N:			1004
R-squared:		0.6544
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  72
Fixed effects by:	authority_code
  No. of FE:		  15
               coeff    se       t   p>t  CI_low CI_high
fpsb_auction   9.461 0.726  13.023 0.000   8.013  10.910
reserve_price  0.000 0.000   5.768 0.000   0.000   0.000
OG02           0.625 0.734   0.851 0.398  -0.839   2.088
OG03          -0.191 0.579  -0.330 0.742  -1.346   0.963
OG04          -4.804 3.505  -1.371 0.175 -11.792   2.184
OG06           0.809 0.754   1.073 0.287  -0.695   2.312
OG07           4.650 5.326   0.873 0.386  -5.969  15.270
OG08           0.658 1.786   0.368 0.714  -2.904   4.220
OG10           8.836 0.540  16.367 0.000   7.760   9.913
OG11           5.566 1.127   4.941 0.000   3.320   7.813
OG12           6.070 1.880   3.228 0.002   2.320   9.819
OS01          -1.823 1.452  -1.256 0.213  -4.717   1.071
OS02           9.654 1.545   6.249 0.000   6.573 

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


Dependent variable:	discount
N:			1109
R-squared:		0.6003
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  86
Fixed effects by:	authority_code
  No. of FE:		  18
               coeff    se       t   p>t  CI_low CI_high
fpsb_auction   7.606 0.564  13.493 0.000   6.486   8.727
reserve_price  0.000 0.000   8.234 0.000   0.000   0.000
OG02          -0.514 0.653  -0.786 0.434  -1.813   0.786
OG03          -1.207 0.433  -2.788 0.007  -2.067  -0.346
OG04          -5.591 3.505  -1.595 0.114 -12.559   1.377
OG06          -0.400 0.806  -0.496 0.621  -2.002   1.203
OG07           4.878 3.674   1.328 0.188  -2.427  12.183
OG08           3.172 2.221   1.428 0.157  -1.244   7.588
OG09           2.392 1.363   1.754 0.083  -0.319   5.102
OG10           8.171 0.533  15.329 0.000   7.111   9.231
OG11           4.825 0.931   5.182 0.000   2.974   6.676
OG12           5.622 2.801   2.007 0.048   0.052  11.192
OG13          -2.822 1.365  -2.067 0.042  -5.537 

In [21]:
#table 6
#(:,7)/ co& ctrl_exp
#값 다름

treatment = ['turin_co_sample','turin_pr_sample']
# mutlicollinearity function
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

#iteration for group
for t in treatment:
    df_reg_co = df[(df['post02pre04']==1)&(df[t]==1)&(df['ctrl_exp_'+t] == 1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
    
    #vif cal
    #first, make a column list
    reg_col = []
    for i in work_list:
        reg_col.append(i)
    for j in year_list:
        reg_col.append(j)
    exog_var = ['fpsb_auction','reserve_price','municipality'] #id_auth빼봄
    exog = exog_var + reg_col 


    #check multicollinearity
    X = df_reg_co.loc[:,exog]
    vif = calc_vif(X)
    #print(vif)


    #delete from col list
    for i in range(len(vif)):
        if np.isnan(vif.loc[i, 'VIF']) == True:
            reg_col.remove(vif.loc[i, 'variables'])
        elif vif.loc[i,'VIF'] > 10:
            for j in exog_var:
                if str(vif.loc[i,'variables']) is j and vif.loc[i,'variables'] is not 'fpsb_auction' and vif.loc[i,'variables'] is not 'id_auth':
                    exog_var.remove(vif.loc[i,'variables'])
                
    exog = exog_var + reg_col
    #exog.remove(2000)
    exog.remove('OG01')
    exog.remove('municipality')
    
    exog.remove(2004)

    #print(exog) #check
    print(mt.reg(df_reg_co, 'discount', exog, fe_name = 'authority_code', cluster = 'auth_anno'))

Dependent variable:	discount
N:			620
R-squared:		0.6765
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  42
Fixed effects by:	authority_code
  No. of FE:		  14
               coeff    se      t   p>t CI_low CI_high
fpsb_auction   9.056 0.815 11.115 0.000  7.410  10.701
reserve_price  0.000 0.000  5.220 0.000  0.000   0.000
OG02           0.547 0.917  0.596 0.554 -1.306   2.399
OG03          -0.517 0.709 -0.729 0.470 -1.949   0.916
OG04          -0.954 0.746 -1.280 0.208 -2.460   0.551
OG06           0.191 1.063  0.179 0.859 -1.957   2.338
OG07          10.069 5.915  1.702 0.096 -1.877  22.015
OG08           3.234 2.518  1.284 0.206 -1.851   8.319
OG10           9.157 0.839 10.908 0.000  7.462  10.852
OG11           5.005 1.548  3.233 0.002  1.878   8.131
OG12           2.830 1.583  1.788 0.081 -0.366   6.026
OS01          -3.600 0.674 -5.340 0.000 -4.961  -2.238
OS02          11.244 0.744 15.107 0.000  9.741  12.747
OS06           5.194 

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


Dependent variable:	discount
N:			720
R-squared:		0.6226
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  51
Fixed effects by:	authority_code
  No. of FE:		  17
               coeff    se      t   p>t CI_low CI_high
fpsb_auction   7.474 0.776  9.631 0.000  5.916   9.033
reserve_price  0.000 0.000  6.601 0.000  0.000   0.000
OG02          -0.783 0.755 -1.038 0.304 -2.300   0.733
OG03          -1.390 0.512 -2.714 0.009 -2.418  -0.361
OG04          -1.241 0.651 -1.905 0.063 -2.549   0.067
OG06          -0.900 1.196 -0.753 0.455 -3.303   1.503
OG07           8.972 6.172  1.454 0.152 -3.425  21.370
OG08           5.078 3.524  1.441 0.156 -2.000  12.156
OG09           0.571 0.769  0.742 0.461 -0.974   2.115
OG10           8.403 0.754 11.152 0.000  6.890   9.917
OG11           4.455 1.042  4.274 0.000  2.361   6.549
OG12          -0.047 0.886 -0.053 0.958 -1.827   1.733
OG13          -4.613 0.765 -6.027 0.000 -6.151  -3.076
OS01          -2.728 